# Pré-processamento dos Dados do CPGF

**Projeto:** Detecção de Anomalias no Uso do Cartão de Pagamento do Governo Federal  
**Disciplina:** Mineração de Dados  
**Etapa:** 1.Pré-processamento

---

## 0. Configuração do Ambiente

Montagem do Google Drive para execução no Colab e importação das bibliotecas necessárias.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

## 1. Carregamento dos Dados

Leitura dos arquivos CSV do CPGF (a partir de 2022) com link disponível na pasta `dados/`.

> O link é um redirecionamento para o Drive com 36 datasets.

In [ ]:
import pandas as pd
import os

caminho = '/content/drive/MyDrive/Projeto_MD_CPGF_inicial/datasets/cpgf_datasets'

arquivos = [f for f in os.listdir(caminho) if f.endswith('.csv')]

lista_dfs = []

for arquivo in arquivos:
    caminho_completo = os.path.join(caminho, arquivo)

    df_temp = pd.read_csv(
        caminho_completo,
        sep=';',
        decimal=',',
        encoding='latin1'
    )

    lista_dfs.append(df_temp)

# juntar tudo
df = pd.concat(lista_dfs, ignore_index=True)

print("Total de linhas:", len(df))
df.head()

---

## 2. Exploração dos Dados
### 2.1 Fundamentação Teórica

Utilizamos as seguintes técnicas e ferramentas:
- altermanos a coluna VALOR TRANSAÇÃO para float.

- Estatísticas resumidas (média, mediana, desvio padrão, quartis);
- Visualizações gráficas  (boxplot nessa etapa)  para identificar
  distribuições, padrões e possíveis anomalias visuais nos atributos.

Essa parte foi necessaria antes de aplicar qualquer técnica de mineração 

In [ ]:
# --- 2.2 Visão geral do DataFrame ---
print(' Primeiras linhas ')
display(df.head())

print('\n Informações gerais')
df.info()

print('\n Tipos de dados ')
display(df.dtypes)


In [ ]:
df['VALOR TRANSAÇÃO'] = pd.to_numeric(df['VALOR TRANSAÇÃO'], errors='coerce')

In [ ]:
# --- 2.3 Estatísticas descritivas ---
print(' Estatísticas descritivas (atributos numéricos) ')
display(df.describe())

print('\n Estatísticas descritivas (atributos categóricos) ')
display(df.describe(include='object'))

In [ ]:
# --- 2.4 Visualizações exploratórias ---

import plotly.express as px
fig = px.violin(df, x="ANO EXTRATO", y="VALOR TRANSAÇÃO", box=True)

fig.update_layout(
    width=1000,
    height=600,
    template="plotly_white",
    xaxis_title="ANO EXTRATO",
    yaxis_title="VALOR TRANSAÇÃO (R$)",
)

fig.show()

---

## 3. Qualidade dos Dados

Os principais problemas tratados nesta etapa são:

#### 3.1. Tratamento de Ausência de Valores

Valores ausentes podem surgir por falhas de coleta ou campos opcionais.
A estratégia padrão inclui eliminar as linhas com valores ausentes em atributos críticos.
a principio, nao eliminamos pois nao usaremos as colunas que contem dados nulos no modelo.

(*discutir depois aula 2 qualidade dos dados)

#### 3.2 Tratamento de Ruído e Dados Inconsistentes

Para objetos, ruído é um objeto estranho.
Para atributos, ruído refere-se à modificação dos valores originais.
entendemos que ruído pode confundir o algoritmo.

(verificar se saque pode ser um ruido)

In [ ]:
# --- 3.2 Análise de valores ausentes ---
print('Valores ausentes por coluna')
df.isna().sum()

In [ ]:
# --- 3.3 Tratamento de valores ausentes ---

# Estratégia 1: Eliminar colunas com mais de X% de nulos
LIMIAR_NULOS = 70  # percentual
colunas_excluir = resumo_nulos[resumo_nulos['% do Total'] > LIMIAR_NULOS].index.tolist()
print(f'Colunas removidas (> {LIMIAR_NULOS}% nulos): {colunas_excluir}')
df.drop(columns=colunas_excluir, inplace=True, errors='ignore')

# Estratégia 2: Para colunas numéricas restantes, imputar com mediana
cols_numericas = df.select_dtypes(include=[np.number]).columns
for col in cols_numericas:
    if df[col].isnull().sum() > 0:
        mediana = df[col].median()
        df[col].fillna(mediana, inplace=True)
        print(f'  Coluna "{col}": nulos imputados com mediana = {mediana:.2f}')

# Estratégia 3: Para colunas categóricas, imputar com moda ou 'DESCONHECIDO'
cols_categoricas = df.select_dtypes(include='object').columns
for col in cols_categoricas:
    if df[col].isnull().sum() > 0:
        df[col].fillna('DESCONHECIDO', inplace=True)
        print(f'  Coluna "{col}": nulos preenchidos com "DESCONHECIDO"')

print(f'\nValores ausentes restantes: {df.isnull().sum().sum()}')

In [ ]:
# --- 3.4 Remoção de ruído e dados inconsistentes ---

# TODO: Ajustar conforme as colunas reais do CSV
# Exemplo: remover transações com valor = 0 (sem significado)
# col_valor = 'VALOR_TRANSACAO'
# n_antes = len(df)
# df = df[df[col_valor] != 0]
# print(f'Registros com valor 0 removidos: {n_antes - len(df)}')

# Nota: Valores negativos podem representar estornos legítimos ou erros.
# Avaliar e documentar a decisão:
# n_negativos = (df[col_valor] < 0).sum()
# print(f'Transações com valor negativo: {n_negativos}')
# Decisão: manter valores negativos para análise ou remover como ruído?

print('Etapa de remoção de ruído concluída.')
print(f'Shape atual do DataFrame: {df.shape}')

#### 3.5 Verificação de Duplicatas

Verificação de registros duplicados na base de dados conforme solicitado.

In [ ]:
# --- 3.5 Verificação de duplicatas ---
df[df.duplicated()]
print("Total de linhas:", len(df))
print("Duplicadas:", df.duplicated().sum())
df[df.duplicated(keep=False)].head()

---

## 4. Agregação dos Dados

### 4.1 Fundamentação

Nesta etapa, criamos um novo dataframe agrupando os dados por CPF do portador do cartão. O objetivo é transformar os dados  em um
perfil de comportamento por portador do cartão, bisando que os algoritmos de mineração identifiquem padrões e anomalias do usuário.


In [ ]:
# --- 4.2 Agregação por CPF do portador ---

df_servidor = (
    df.groupby('CPF PORTADOR')
      .agg(
          total_gasto=('VALOR TRANSAÇÃO', 'sum'),
          media_gasto=('VALOR TRANSAÇÃO', 'mean'),
          qtd_transacoes=('VALOR TRANSAÇÃO', 'count'),
          max_gasto=('VALOR TRANSAÇÃO', 'max')
      )
      .reset_index()
)

In [ ]:
df_servidor.head()

---

## 5. Transformação dos Dados
### 5.1 Normalização

Técnicas comuns de normalização:

- **Min-Max Scaling:** Transforma os valores para o intervalo [0, 1].
- **Z-Score (Standardization):** Centraliza na média 0 e desvio padrão 1.

Utilizaremos StandardScaler (Z-Score) neste projeto, pois é mais robusto
na presença de outliers que a normalização Min-Max.

In [ ]:
from sklearn.preprocessing import StandardScaler

features = ['total_gasto', 'media_gasto', 'qtd_transacoes', 'max_gasto']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_servidor[features])

In [ ]:
df_scaled = pd.DataFrame(
    X_scaled,
    columns=features,
    index=df_servidor.index
)

In [ ]:
df_scaled.head()

---

## 6. Exportação dos Dados Pré-processados

O DataFrame limpo e normalizado é salvo para ser consumido pelo Notebook 02 (Mineração).

In [ ]:
# --- 6.1 Salvar dados processados ---
OUTPUT_DIR = os.path.join('..', 'dados')
# Para Colab:
# OUTPUT_DIR = '/content/drive/MyDrive/cpgf-anomaly-detection/dados'

# DataFrame original limpo (sem normalização) — para análise qualitativa no Notebook 03
df.to_csv(os.path.join(OUTPUT_DIR, 'cpgf_limpo.csv'), index=False, sep=';', encoding='utf-8')
print('Arquivo salvo: cpgf_limpo.csv')

# DataFrame normalizado — para entrada nos algoritmos de agrupamento
df_normalizado.to_csv(os.path.join(OUTPUT_DIR, 'cpgf_normalizado.csv'), index=False, sep=';', encoding='utf-8')
print('Arquivo salvo: cpgf_normalizado.csv')

print(f'\nShape final: {df_normalizado.shape}')
print('\n✅ Pré-processamento concluído com sucesso!')